<a href="https://colab.research.google.com/github/saibala2905/Pokedex_T5_Engine/blob/main/Pok%C3%A9dex_T5_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install accelerate -U
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!git clone https://huggingface.co/saibala29/Pokedex_T5_Engine

Cloning into 'Pokedex_T5_Engine'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 1.12 KiB | 1.12 MiB/s, done.


In [5]:
%cd /content/Pokedex_T5_Engine

/content/Pokedex_T5_Engine


In [10]:
pwd

'/content/Pokedex_T5_Engine'

## Import Libraries


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

## Load Dataset

In [7]:
data_path = '/content/Pokedex_T5_Engine/Pokemon_Final_Fixed_Questions_Queries.csv'  # Update this path
data = pd.read_csv(data_path)

In [8]:
data.head(10)

,question,query
0,Retrieve all Pokemon,db.pokemon_stats.find({})
1,"Find all Pokemon of a specific type, e.g., Fire","db.pokemon_stats.find({""Type 1"": ""Fire""})"
2,Find all Legendary Pokemon,"db.pokemon_stats.find({""Legendary"": true})"
3,"Find all Pokemon from a specific generation, e...","db.pokemon_stats.find({""Generation"": 1})"
4,Find Pokemon with HP greater than 100,"db.pokemon_stats.find({""HP"": {$gt: 100}})"
5,Sort Pokemon by Attack in descending order,"db.pokemon_stats.find({}).sort({""Attack"": -1})"
6,Find all Water-type Pokemon that are also Lege...,"db.pokemon_stats.find({""Type 1"": ""Water"", ""Leg..."
7,Count the number of Pokemon of each type,"db.pokemon_stats.aggregate([{$group: {_id: ""$T..."
8,Find all Pokemon that do not have a secondary ...,"db.pokemon_stats.find({""Type 2"": null})"
9,"Find Pokemon with a specific name, e.g., Pikachu","db.pokemon_stats.find({""Name"": ""Pikachu""})"


In [9]:
# Preprocess the data
def preprocess_data(data):
    data['input_text'] = "question: " + data['question']
    data['target_text'] = data['query']
    return data

data = preprocess_data(data)

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1)

# Tokenization
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize(batch):
    tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_target = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=512)

    return {
        'input_ids': tokenized_input.input_ids,
        'attention_mask': tokenized_input.attention_mask,
        'labels': tokenized_target.input_ids
    }

train_data = train_data.apply(tokenize, axis=1)
val_data = val_data.apply(tokenize, axis=1)

class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        return {key: torch.tensor(val) for key, val in item.items()}

train_dataset = T5Dataset(train_data)
val_dataset = T5Dataset(val_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Fine Tune Model

In [11]:
# Model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,  # Increase epochs as needed
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Save the model
model.save_pretrained('./T5_fine_tuned')
tokenizer.save_pretrained('./t5_fine_tuned')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step,Training Loss
10,13.905600
20,14.530000
30,14.650300
40,13.870100
50,12.786600
60,12.032900
70,11.679700
80,10.165200
90,8.501600
100,7.305100


('./t5_fine_tuned/tokenizer_config.json',
 './t5_fine_tuned/special_tokens_map.json',
 './t5_fine_tuned/spiece.model',
 './t5_fine_tuned/added_tokens.json')

In [18]:
# Load the trained model and tokenizer
model_path = './T5_fine_tuned'
tokenizer_path = './t5_fine_tuned'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

def generate_query(question, model, tokenizer):
    # Prepend 'question:' to the input question
    input_text = "question: " + question
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=True)
    # Generate the output (query)
    output_ids = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    # Decode and return the generated text
    query = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return query

# Example usage
question = "Find all electric type pokemon"
query = generate_query(question, model, tokenizer)
print(query)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


db.pokemon_stats.find("Type 1": "Electric")


In [20]:
!git config user.email "saibalalearning@gmail.com"
!git config user.name "saibala29"

In [21]:
!git add .
!git commit -m "Add fine-tuned Pokedex_T5_Engine"
!git push

[main b8837ab] Add fine-tuned Pokedex_T5_Engine
 33 files changed, 3516 insertions(+)
 create mode 100644 Pokemon_Final_Fixed_Questions_Queries.csv
 create mode 100644 T5_fine_tuned/config.json
 create mode 100644 T5_fine_tuned/generation_config.json
 create mode 100644 T5_fine_tuned/model.safetensors
 create mode 100644 logs/events.out.tfevents.1709115409.aac79a2bc2a9.444.0
 create mode 100644 results/checkpoint-1000/config.json
 create mode 100644 results/checkpoint-1000/generation_config.json
 create mode 100644 results/checkpoint-1000/model.safetensors
 create mode 100644 results/checkpoint-1000/optimizer.pt
 create mode 100644 results/checkpoint-1000/rng_state.pth
 create mode 100644 results/checkpoint-1000/scheduler.pt
 create mode 100644 results/checkpoint-1000/trainer_state.json
 create mode 100644 results/checkpoint-1000/training_args.bin
 create mode 100644 results/checkpoint-1500/config.json
 create mode 100644 results/checkpoint-1500/generation_config.json
 create mode 1006